# Plot COPSE forcings

In [ ]:
#Configure width of Jupyter notebook in browser
#https://discourse.julialang.org/t/decrease-margins-of-ijulia-in-jupyter-notebook/6912
# display("text/html", "<style>.container { width:100% !important; }</style>")
#Increase width used by Julia output eg DataFrame
ENV["COLUMNS"]=300
ENV["LINES"]=1000

# activate the correct environment
import Pkg
# assuming we are in PALEOcopse/test/forcings, activate PALEOcopse environment
Pkg.activate(joinpath(pwd(), "../../"))

# activate the test environment (for additional packages, plots etc)
using TestEnv
TestEnv.activate()

using Plots

In [ ]:

import PALEOboxes as PB
import PALEOmodel
import PALEOcopse

using DataFrames

# using StatsPlots



In [ ]:
model = PB.create_model_from_config(joinpath(PALEOcopse.moduledir(), "test/forcings/configforcings.yaml"), "model1")

In [ ]:
modeldata = PB.create_modeldata(model)
PB.allocate_variables!(model, modeldata)

PALEOmodel.set_default_solver_view!(model, modeldata)

@PB.TestUtils.check_true PB.check_ready(model, modeldata)

In [ ]:
# Get global Domain
global_domain = PB.get_domain(model, "global")

# Create a vector of all Variables in global Domain (these will include 'tforce' and the forcing variables)
global_vars = PB.get_variables(global_domain)
println("global Domain Variables: ", [v.name for v in global_vars])

In [ ]:
# Initialize model Reactions (will read forcing data from files etc)
PB.initialize_reactiondata!(model, modeldata)

PB.dispatch_setup(model, :setup, modeldata)

dispatchlists = modeldata.dispatchlists_all


In [ ]:
# check force_LIPs has expected default_lambda
global_domain = PB.get_domain(model, "global")
force_LIPs = PB.get_reaction(global_domain, "force_LIPs")

@PB.TestUtils.check_true round(force_LIPs.default_lambda, sigdigits=6) == 1.11528e-08

In [ ]:
# Iterate through time and create DataFrame with all forcing variables

df = DataFrame()
for v in global_vars                   
    insertcols!(df, DataFrames.ncol(df)+1, Symbol(v.name) => Vector{eltype(PB.get_data(v, modeldata)[])}())
end

for tforce in range(-700e6, 100e6, step=1e6)
    PB.set_tforce!(modeldata.solver_view_all, tforce)
    PB.do_deriv(dispatchlists)
    # DataFrame needs  Dict of name_as_symbol => scalar value
    scalar_data = Dict(Symbol(v.name) => PB.get_data(v, modeldata)[] for v in global_vars)
    # Add row to DataFrame
    push!(df, scalar_data)
end


In [ ]:
# Show complete table

df

In [ ]:
# Plot normalized forcings
fieldnames = ["DEGASS_haq_copsereloaded", "DEGASS_bergman2004"]

# plot requires a Matrix with data for lines in columns, and a 1xn Matrix for fieldnames
# plotdata = hcat([df[!, fn] for fn in fieldnames]...)
plotdata = Matrix(df[!, fieldnames])
labeldata = hcat(fieldnames...)  # convert Vector{String} to Array{String,2}

plot(df.tforce, plotdata, labels=labeldata)

In [ ]:
# Show table of values
df[!, ["tforce", fieldnames...]]

In [ ]:
# Plot normalized forcings
fieldnames = ["UPLIFT_berner_fr", "UPLIFT_bergman2004"]

# plot requires a Matrix with data for lines in columns, and a 1xn Matrix for fieldnames
# plotdata = hcat([df[!, fn] for fn in fieldnames]...)
plotdata = Matrix(df[!, fieldnames])
labeldata = hcat(fieldnames...)  # convert Vector{String} to Array{String,2}

plot(df.tforce, plotdata, labels=labeldata)

In [ ]:
# Show table of values
df[!, ["tforce", fieldnames...]]

In [ ]:
# Plot normalized forcings
fieldnames = ["EVO", "W","Bforcing", "CPland_relative"]

# plot requires a Matrix with data for lines in columns, and a 1xn Matrix for fieldnames
# plotdata = hcat([df[!, fn] for fn in fieldnames]...)
plotdata = Matrix(df[!, fieldnames])
labeldata = hcat(fieldnames...)  # convert Vector{String} to Array{String,2}

plot(df.tforce, plotdata, labels=labeldata)

In [ ]:
# Show table of values
df[!, ["tforce", fieldnames...]]

In [ ]:
# Plot normalized forcings
fieldnames = ["CAL_NORM", "ORGEVAP_AREA", "GRAN", "COAL"]

# plot requires a Matrix with data for lines in columns, and a 1xn Matrix for fieldnames
# plotdata = hcat([df[!, fn] for fn in fieldnames]...)
plotdata = Matrix(df[!, fieldnames])
labeldata = hcat(fieldnames...)  # convert Vector{String} to Array{String,2}

plot(df.tforce, plotdata, labels=labeldata)

In [ ]:
# Show table of values
df[!, ["tforce", fieldnames...]]

In [ ]:
# Plot normalized forcings
fieldnames = ["CFB_area"]

# plot requires a Matrix with data for lines in columns, and a 1xn Matrix for fieldnames
# plotdata = hcat([df[!, fn] for fn in fieldnames]...)
plotdata = Matrix(df[!, fieldnames])
labeldata = hcat(fieldnames...)  # convert Vector{String} to Array{String,2}

plot(df.tforce, plotdata, labels=labeldata)

In [ ]:
# Show table of values
df[!, ["tforce", fieldnames...]]